In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
gdf = gpd.read_feather(r"C:\Users\Pedro\Outer Loop Consulting\2022-01 - TMR GSM - Documents\General\submodels\logistic_path_choice\data\compass_iot_traces\all_vertices_sample.feather")

In [3]:
gdf.head()

,vehicle_id,trip_id,index,timestamp,vertex_raw,snap_excluded,timestamp_interval_secs,distance_interval_m,inferred_speed_kmh,turning_angle,unrealistic_g_force
0,EXYKWC93lRl9ZD78kBXVhA==,0BqjUTPOuTw+tw5zYGHHVA==,10,2023-01-08 09:19:24,POINT (153.42824 -28.00981),None,40.0,208.113186,18.730187,164.0,None
1,EXYKWC93lRl9ZD78kBXVhA==,0BqjUTPOuTw+tw5zYGHHVA==,11,2023-01-08 09:20:07,POINT (153.42989 -28.01309),None,43.0,398.404526,33.354798,150.0,None
2,EXYKWC93lRl9ZD78kBXVhA==,0BqjUTPOuTw+tw5zYGHHVA==,12,2023-01-08 09:23:39,POINT (153.42914 -28.02482),None,212.0,1301.191993,22.095713,NaN,None
3,VVQGAY+OI5RC2Siv0yyfmA==,0br3frDC4JmToasd9QVTJQ==,2,2023-01-10 20:48:02,POINT (153.39049 -27.97962),None,274.0,11.857542,0.155793,141.0,None
4,VVQGAY+OI5RC2Siv0yyfmA==,0br3frDC4JmToasd9QVTJQ==,3,2023-01-10 20:48:18,POINT (153.39067 -27.97945),None,16.0,25.855514,5.817491,118.0,None


In [4]:
gdf["posixtime"] = (gdf.timestamp - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

In [11]:
df = pd.DataFrame({"posixtime":gdf.posixtime, 
                               "x": gdf.geometry.x,
                               "y": gdf.geometry.y})
agg = df.groupby(["posixtime"]).agg(["min", "max", "mean", "count"])
            

In [16]:
jitter = np.sqrt((agg.x["min"] - agg.x["max"]) ** 2 + (agg.y["min"] - agg.y["max"]) ** 2) / 1000
repeated = agg[agg.x["count"]>1]


In [17]:
repeated

x                                        y             \
                   min         max        mean count        min        max   
posixtime                                                                    
1672857014  149.159683  153.063675  151.111679     2 -27.323027 -21.102890   
1672858660  152.951706  153.097336  153.024521     2 -27.646200 -27.176432   
1672859130  153.001968  153.096680  153.049324     2 -27.646990 -27.228760   
1672859628  152.874060  153.014282  152.944171     2 -27.642670 -27.599717   
1672859648  152.875214  153.295312  153.085263     2 -27.619461 -27.600529   
...                ...         ...         ...   ...        ...        ...   
1674816618  153.006756  153.281326  153.144041     2 -27.583423 -27.436632   
1674816648  153.006953  153.285095  153.146024     2 -27.584074 -27.436991   
1674816738  153.007725  153.296707  153.152216     2 -27.582987 -27.444571   
1674817390  146.816444  153.039872  149.928158     2 -27.462141 -19.259012   
1674823635  152.883545  153.078510  152.981028     2 -27.566231 -26.660112   

                             
                 mean count  
posixtime                    
1672857014 -24.212958     2  
1672858660 -27.411316     2  
1672859130 -27.437875     2  
1672859628 -27.621193     2  
1672859648 -27.609995     2  
...               ...   ...  
1674816618 -27.510028     2  
1674816648 -27.510532     2  
1674816738 -27.513779     2  
1674817390 -23.360577     2  
1674823635 -27.113171     2  

[673749 rows x 8 columns]